<a href="https://colab.research.google.com/github/hanaessamm/CodeAlpha_FAQ_Chatbot/blob/main/FAQ_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q sentence-transformers


In [ ]:
from google.colab import files
import io
import pandas as pd

uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['faq_dataset.csv']))

questions = df['Question'].tolist()
answers = df['Answer'].tolist()


Saving faq_dataset.csv to faq_dataset.csv


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# turn all FAQ questions into vectors so we can compare later
question_embeddings = model.encode(questions, convert_to_tensor=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Function that gets the best answer for what the user asks

def get_response(user_input):

    # converting the user's question into a vector (embedding)..
    query_embedding = model.encode(user_input, convert_to_tensor=True)

    # comparing the user's question with all stored questions using cosine similarity
    scores = util.pytorch_cos_sim(query_embedding, question_embeddings)[0]

    # find the index of the most similar question (highest score).
    best_idx = scores.argmax().item()
    best_score = scores[best_idx].item()  # the actual similarity value of the best match.

    if best_score < 0.6:
        return "I'm not sure I understand. Could you try rephrasing your question?"

    return answers[best_idx]


In [ ]:
print("Welcome to the FAQ Chatbot! Type 'exit' to quit.\n")

while True:
    user_input = input("You: ")
    if user_input.lower() == 'exit':
        print("Bot: Goodbye!")
        break
    response = get_response(user_input)
    print("Bot:", response)


Welcome to the FAQ Chatbot! Type 'exit' to quit.

You: what sizes do you offer?
Bot: We offer sizes ranging from XS to XL. You can find detailed measurements in our Size Chart.
You: do you have a size chart?
Bot: Yes, we have a Size Chart available on every product page for easy reference.
You: How long does delivery take?
Bot: Standard shipping usually takes 5-10 business days, depending on your location.
You: What is your return policy?
Bot: We accept returns within 14 days of delivery, provided items are unworn, unwashed, and in original condition.
You: Are your clothes true to size?
Bot: Our garments are designed to fit true to size. If you prefer a looser fit, consider sizing up.
You: Do you sell Loafers?
Bot: I'm not sure I understand. Could you try rephrasing your question?
You: Do you have modest wear?
Bot: Yes, our collections are designed with modest, elegant styles in mind.
You: exit
Bot: Goodbye!
